In [103]:
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import mobilenet_v2
from keras.layers import Input
from keras.models import Model
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [11]:
base_dir='/content/sample_data/data'

# load data 

In [73]:
datagen=ImageDataGenerator(rescale=1./255,rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
imageGeneratir=datagen.flow_from_directory(base_dir,target_size=(224,224),
                                           batch_size=32,class_mode='binary')

Found 3226 images belonging to 3 classes.


In [91]:
baseModel = mobilenet_v2.MobileNetV2(weights="imagenet", include_top=False,input_shape=(224,224,3))

In [94]:
 batch_size=32
 def extract_feature(directory,sample_count):
  feature=np.zeros(shape=(sample_count,7,7,1280))
  labels=np.zeros(shape=(sample_count))
  imageGenerator=datagen.flow_from_directory(directory,
                                             batch_size=batch_size,target_size=(224,224),class_mode='binary')
  i=0
  for image , label in imageGenerator:

    if image.shape[0]<batch_size:
      continue
    F=baseModel.predict(image)

    feature[i*batch_size:(i+1)*batch_size]=F
    labels[i*batch_size:(i+1)*batch_size]=label
    i+=1
    if i*batch_size>sample_count:
      break
    return feature, labels

In [98]:
features,labels=extract_feature(base_dir,3226)


Found 3226 images belonging to 3 classes.


In [101]:
dim=features.shape[1]*features.shape[2]*features.shape[3]
features=np.reshape(features,newshape=(3226,dim))

In [106]:
train_feature,test_feature,train_label,test_label=train_test_split(features,labels,test_size=0.25,random_state=0)

# creat model

In [102]:
from keras.optimizers import adam_v2
Adam=adam_v2.Adam(lr=1e-4, decay=1e-4 /20)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [110]:
input=Input(shape=(62720))
# headModel = layers.AveragePooling2D(pool_size=(7, 7))(input)
# headModel = layers.Flatten(name="flatten")(headModel)
headModel = layers.Dense(128, activation="relu")(input)
headModel = layers.Dropout(0.25)(headModel)
headModel=layers.Dense(64,activation='relu')(headModel)
headModel=layers.Dropout(0.25)(headModel)
headModel=layers.Dense(32,activation='relu')(headModel)
headModel = layers.Dense(1, activation="sigmoid")(headModel)
model = Model(input,headModel)

model.compile(loss="binary_crossentropy",optimizer=Adam,metrics=['acc'])

# Fit model 

In [112]:
history=model.fit(train_feature,train_label,batch_size=32,epochs=20,validation_split=0.2)

Epoch 1/20
61/61 [==============================] - 3s 49ms/step - loss: -1.7937 - acc: 0.9953 - val_loss: -3.9241 - val_acc: 0.9917
Epoch 2/20
61/61 [==============================] - 3s 49ms/step - loss: -2.7564 - acc: 0.9953 - val_loss: -5.5792 - val_acc: 0.9917
Epoch 3/20
61/61 [==============================] - 3s 49ms/step - loss: -3.5143 - acc: 0.9953 - val_loss: -7.4521 - val_acc: 0.9917
Epoch 4/20
61/61 [==============================] - 3s 49ms/step - loss: -5.1691 - acc: 0.9953 - val_loss: -9.4719 - val_acc: 0.9917
Epoch 5/20
61/61 [==============================] - 3s 49ms/step - loss: -6.4761 - acc: 0.9953 - val_loss: -12.7566 - val_acc: 0.9917
Epoch 6/20
61/61 [==============================] - 3s 48ms/step - loss: -8.7628 - acc: 0.9953 - val_loss: -16.6084 - val_acc: 0.9917
Epoch 7/20
61/61 [==============================] - 3s 49ms/step - loss: -12.3651 - acc: 0.9953 - val_loss: -22.4553 - val_acc: 0.9917
Epoch 8/20
61/61 [==============================] - 3s 49ms/step 

In [116]:
predIdxs = model.predict(test_feature, batch_size=32)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(test_label, predIdxs))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       796
         1.0       0.00      0.00      0.00         9
         2.0       0.00      0.00      0.00         2

    accuracy                           0.99       807
   macro avg       0.33      0.33      0.33       807
weighted avg       0.97      0.99      0.98       807



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [117]:
model.save('/content/sample_data/my_model_2.h5')